In [14]:
import numpy as np
import netCDF4 as nc
import glob
import matplotlib.pyplot as plt
import re
import pandas as pd

In [15]:
file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/'

file_list = sorted(glob.glob('/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/2023*.nc'))  # Adjust path based on month


In [16]:
mmdd_list = []
radarAccum_list = []

for file in file_list:
    dayFile = nc.Dataset(file)

    dbz = np.array(dayFile.variables['Zh'][:, 0])
    dbz[dbz > 1000] = np.nan
    z = 10 ** (dbz / 10)

    a = 18
    b = 1.1

    precip = (z / a) ** (1 / b)

    radarAccum = np.trapz(precip[~np.isnan(precip)], dx=24/len(precip))
    
    # Regular expression to find MMDD after '2023'
    match = re.search(r'/2023(\d{4})', file)
    mmdd = match.group(1)

    mmdd_list.append(mmdd)
    radarAccum_list.append(radarAccum)




/tmp/ipykernel_2432398/2987545486.py:16: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  radarAccum = np.trapz(precip[~np.isnan(precip)], dx=24/len(precip))


In [17]:
df = pd.DataFrame({'mmdd': mmdd_list, 'radarAccum': radarAccum_list})
df.to_csv('accumulation_by_day.csv', index=False)